In [1]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install mpi4py
!pip install stable-baselines[mpi]

     |████████████████████████████████| 1.4MB 7.3MB/s 
  Created wheel for mpi4py: filename=mpi4py-3.0.3-cp36-cp36m-linux_x86_64.whl size=2074477 sha256=cb94524663ff6f0414553557bf367d84f1470bb41e06b512e60bea6690cce160
  Stored in directory: /root/.cache/pip/wheels/18/e0/86/2b713dd512199096012ceca61429e12b960888de59818871d6
Successfully built mpi4py
     |████████████████████████████████| 256kB 7.7MB/s 


In [1]:
from stable_baselines import SAC
from stable_baselines.gail import generate_expert_traj

# Generate expert trajectories (train expert)
model = SAC('MlpPolicy', 'Pendulum-v0', verbose=1)
# Train for 60000 timesteps and record 10 trajectories
# all the data will be saved in 'expert_pendulum.npz' file
generate_expert_traj(model, 'expert_pendulum', n_timesteps=6000, n_episodes=10)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Creating environment from the given name, wrapped in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


-----------------------------------------
| current_lr              | 0.0003      |
| ent_coef                | 0.8608094   |
| ent_coef_loss           | -0.24633104 |
| entropy                 | 1.2327181   |
| episodes                | 4           |
| fps                     | 266         |
| mean 100 episode reward | -1.31e+03   |
| n_updates               | 501         |
| policy_loss             | 13.137248   |
| qf1_loss                | 0.9311091   |
| qf2_loss                | 0.75123173  |
| time_elapsed            | 2           |
| total timesteps         | 600         |
| va

{'actions': array([[ 1.5965114],
        [ 1.79791  ],
        [ 1.8551197],
        ...,
        [-0.9501265],
        [-1.2492963],
        [-1.3843004]], dtype=float32),
 'episode_returns': array([-1331.68366033, -1347.86064103,  -977.88751299, -1323.92821711,
        -1063.91554219, -1275.46926838, -1065.75622757, -1164.60572511,
        -1287.84722716, -1197.40907136]),
 'episode_starts': array([ True, False, False, ..., False, False, False]),
 'obs': array([[-9.9410349e-01, -1.0843533e-01, -2.4793188e-01],
        [-9.9458027e-01, -1.0397164e-01, -8.9781679e-02],
        [-9.9403745e-01, -1.0903896e-01,  1.0192609e-01],
        ...,
        [ 9.9999523e-01,  3.0951523e-03,  4.9732761e+00],
        [ 9.7019815e-01,  2.4231291e-01,  4.8330789e+00],
        [ 8.8415223e-01,  4.6719894e-01,  4.8274188e+00]], dtype=float32),
 'rewards': array([-9.20744324, -9.23003674, -9.19954491, ..., -2.47426009,
        -2.3973279 , -2.5686264 ])}

In [2]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 95kB/s 
     |████████████████████████████████| 3.2MB 26.7MB/s 
     |████████████████████████████████| 491kB 41.3MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [2]:
import gym

from stable_baselines import GAIL, SAC
from stable_baselines.gail import ExpertDataset, generate_expert_traj

# Generate expert trajectories (train expert)
model = SAC('MlpPolicy', 'Pendulum-v0', verbose=1)
generate_expert_traj(model, 'expert_pendulum', n_timesteps=100, n_episodes=10)

# Load the expert dataset
dataset = ExpertDataset(expert_path='expert_pendulum.npz', traj_limitation=10, verbose=1)

model = GAIL('MlpPolicy', 'Pendulum-v0', dataset, verbose=1)
# Note: in practice, you need to train for 1M steps to have a working policy
model.learn(total_timesteps=1000)
model.save("gail_pendulum")

del model # remove to demonstrate saving and loading

model = GAIL.load("gail_pendulum")

env = gym.make('Pendulum-v0')
obs = env.reset()
while True:
  action, _states = model.predict(obs)
  obs, rewards, dones, info = env.step(action)
  env.render()

Creating environment from the given name, wrapped in a DummyVecEnv.
actions (2000, 1)
obs (2000, 3)
rewards (2000,)
episode_returns (10,)
episode_starts (2000,)
actions (2000, 1)
obs (2000, 3)
rewards (2000,)
episode_returns (10,)
episode_starts (2000,)
Total trajectories: 10
Total transitions: 2000
Average returns: -1559.6456991951914
Std for returns: 223.75969478737093
Creating environment from the given name, wrapped in a DummyVecEnv.
********** Iteration 0 ************
Optimizing Policy...
sampling
done in 1.367 seconds
computegrad
done in 0.129 seconds
conjugate_gradient
      iter residual norm  soln norm
         0    0.00917          0
         1     0.0096     0.0163
         2   0.000407     0.0654
         3   0.000209      0.073
         4   0.000168     0.0956
         5   9.72e-05      0.169
         6   0.000597      0.199
         7   0.000233      0.207
         8   1.77e-05       0.38
         9   1.93e-06      0.466
        10   0.000158      0.481
done in 0.256 seco

NoSuchDisplayException: ignored